In [1]:
import sys
sys.path.append("../")

In [11]:
from langgraph.prebuilt import create_react_agent
from langchain_core.tools import tool

from app.config import get_llm, get_embeddings, get_vector_store, get_vanna

In [12]:
llm = get_llm()
embedding = get_embeddings()
vector_store = get_vector_store()
vanna = get_vanna()

In [5]:
llm.invoke("Hello, how are you?")

AIMessage(content="I'm just a language model, so I don't have feelings or emotions like humans do, but I'm functioning properly and ready to help with any questions or tasks you have! How can I assist you today?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 16, 'total_tokens': 57, 'completion_time': 0.043815231, 'prompt_time': 0.000225337, 'queue_time': 0.054766923, 'total_time': 0.044040568}, 'model_name': 'meta-llama/llama-4-maverick-17b-128e-instruct', 'system_fingerprint': 'fp_c527aa4474', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--f007b8e7-159c-4967-bcd8-2be08b651446-0', usage_metadata={'input_tokens': 16, 'output_tokens': 41, 'total_tokens': 57})

In [6]:
vector_store.similarity_search("how fraud is committed worldwide?", k=5)

[Document(metadata={'source': '..\\data\\txt\\Understanding Credit Cards Fraud.txt', '_id': '516965eb-da60-4e81-a433-f245b16001b5', '_collection_name': 'my_documents'}, page_content='**OVERVIEW\n\n**\n\nINTRODUCTION .......................................................1\n\nPURPOSE OF THIS PAPER...........................................1\n\nCURRENT STATE OF THE INDUSTRY..............................2\n\nHOW FRAUD IS COMMITTED WORLDWIDE?......................2\n\n**FRAUD TECHNIQUES\n\n**.......................................................3\n\nCARD RELATED FRAUDS.............................................4\n\nAPPLICATION FRAUD...............................................4\n\nLOST/STOLEN CARDS..................................................4\n\nACCOUNT TAKEOVER................................................4\n\nFAKE AND COUNTERFEIT CARDS...............................4\n\nMERCHANT RELATED FRAUDS......................................5\n\nMERCHANT COLLUSION......................

In [14]:
vanna.ask("What is the monthly fraud rate over the last two years?", visualize=False, allow_llm_to_see_data=True, auto_train=False)

SQL Prompt: [{'role': 'system', 'content': 'You are a PostgreSQL expert. Please help to generate a SQL query to answer the question. Your response should ONLY be based on the given context and follow the response guidelines and format instructions. \n===Tables \n\nCREATE TABLE public.fraud_data (\n\ttrans_date_trans_time timestamp NULL,\n\tcc_num int8 NULL,\n\tmerchant text NULL,\n\tcategory text NULL,\n\tamt float8 NULL,\n\t"first" text NULL,\n\t"last" text NULL,\n\tgender text NULL,\n\tstreet text NULL,\n\tcity text NULL,\n\tstate text NULL,\n\tzip int8 NULL,\n\tlat float8 NULL,\n\tlong float8 NULL,\n\tcity_pop int8 NULL,\n\tjob text NULL,\n\tdob text NULL,\n\ttrans_num text NULL,\n\tunix_time int8 NULL,\n\tmerch_lat float8 NULL,\n\tmerch_long float8 NULL,\n\tis_fraud bool NULL\n)\n\n\n\n===Additional Context \n\nThis is a simulated credit card transaction dataset containing legitimate and fraud transactions from the duration 1st Jan 2019 - 31st Dec 2020. It covers credit cards of 10

("WITH bounds AS (\n                SELECT (MAX(trans_date_trans_time) - INTERVAL '2 years') AS start_dt\n                FROM public.fraud_data\n            )\n            SELECT date_trunc('month', trans_date_trans_time)::date AS month,\n                COUNT(*) AS n_tx,\n                SUM((is_fraud)::int) AS fraud_tx,\n                AVG((is_fraud)::int)::numeric(10,4) AS fraud_rate\n            FROM public.fraud_data f\n            JOIN bounds b ON f.trans_date_trans_time >= b.start_dt\n            GROUP BY 1\n            ORDER BY 1;",
         month    n_tx  fraud_tx fraud_rate
 0  2020-06-01   30058       133     0.0044
 1  2020-07-01   85848       321     0.0037
 2  2020-08-01   88759       415     0.0047
 3  2020-09-01   69533       340     0.0049
 4  2020-10-01   69348       384     0.0055
 5  2020-11-01   72635       294     0.0040
 6  2020-12-01  139538       258     0.0018,
 None)

In [15]:
from pydantic import BaseModel, Field


class Query(BaseModel):
    query: str = Field(..., description="The input query to the agent by the user")

In [16]:
@tool(args_schema=Query)
def ask_about_credit_cards_fraud_theory(query: str) -> str:  
    """Useful for for retrieving information about credit card frauds theory."""
    return vector_store.similarity_search(query, k=5)

@tool(args_schema=Query)
def ask_about_credit_cards_fraud_data(query: str) -> str:  
    """Useful for for retrieving information about credit card frauds data."""
    return vanna.ask(query, visualize=False, allow_llm_to_see_data=True, auto_train=False)[1]

agent = create_react_agent(
    model=llm,  
    tools=[ask_about_credit_cards_fraud_theory, ask_about_credit_cards_fraud_data],  
    prompt="You are a helpful assistant"  
)

In [17]:
# Run the agent
response = agent.invoke(
    {"messages": 
        [
            {"role": "user", "content": "how many fraud over the last week?"},
            # {"role": "user", "content": "how fraud is committed worldwide?"}
        ]
    }
)

response['messages'][-1].content

SQL Prompt: [{'role': 'system', 'content': 'You are a PostgreSQL expert. Please help to generate a SQL query to answer the question. Your response should ONLY be based on the given context and follow the response guidelines and format instructions. \n===Tables \n\nCREATE TABLE public.fraud_data (\n\ttrans_date_trans_time timestamp NULL,\n\tcc_num int8 NULL,\n\tmerchant text NULL,\n\tcategory text NULL,\n\tamt float8 NULL,\n\t"first" text NULL,\n\t"last" text NULL,\n\tgender text NULL,\n\tstreet text NULL,\n\tcity text NULL,\n\tstate text NULL,\n\tzip int8 NULL,\n\tlat float8 NULL,\n\tlong float8 NULL,\n\tcity_pop int8 NULL,\n\tjob text NULL,\n\tdob text NULL,\n\ttrans_num text NULL,\n\tunix_time int8 NULL,\n\tmerch_lat float8 NULL,\n\tmerch_long float8 NULL,\n\tis_fraud bool NULL\n)\n\n\n\n===Additional Context \n\nThis is a simulated credit card transaction dataset containing legitimate and fraud transactions from the duration 1st Jan 2019 - 31st Dec 2020. It covers credit cards of 10

'There were 0 reported cases of credit card fraud over the last week.'

In [18]:
# Run the agent
response = agent.invoke(
    {"messages": 
        [
            # {"role": "user", "content": "how many fraud over the last week?"},
            {"role": "user", "content": "how fraud is committed worldwide?"}
        ]
    }
)

response['messages'][-1].content

'Fraud is committed worldwide through various methods, including lost or stolen cards (48%), identity theft (15%), skimming or cloning (14%), counterfeit cards (12%), mail intercept fraud (6%), and other methods (5%). Some of the high-risk countries for credit card fraud include Ukraine (19%), Indonesia (18.3%), Yugoslavia (17.8%), Turkey (9%), and Malaysia (5.9%). The methods used by fraudsters include application fraud, lost or stolen cards, account takeover, fake and counterfeit cards, merchant collusion, and triangulation.'

In [20]:
# Run the agent
response = agent.invoke(
    {"messages": 
        [
            # {"role": "user", "content": "how many fraud over the last week?"},
            {"role": "user", "content": "how fraud is committed worldwide? and how many fraud committed?"}
        ]
    }
)

response['messages'][-1].content

SQL Prompt: [{'role': 'system', 'content': 'You are a PostgreSQL expert. Please help to generate a SQL query to answer the question. Your response should ONLY be based on the given context and follow the response guidelines and format instructions. \n===Tables \n\nCREATE TABLE public.fraud_data (\n\ttrans_date_trans_time timestamp NULL,\n\tcc_num int8 NULL,\n\tmerchant text NULL,\n\tcategory text NULL,\n\tamt float8 NULL,\n\t"first" text NULL,\n\t"last" text NULL,\n\tgender text NULL,\n\tstreet text NULL,\n\tcity text NULL,\n\tstate text NULL,\n\tzip int8 NULL,\n\tlat float8 NULL,\n\tlong float8 NULL,\n\tcity_pop int8 NULL,\n\tjob text NULL,\n\tdob text NULL,\n\ttrans_num text NULL,\n\tunix_time int8 NULL,\n\tmerch_lat float8 NULL,\n\tmerch_long float8 NULL,\n\tis_fraud bool NULL\n)\n\n\n\n===Additional Context \n\nThis is a simulated credit card transaction dataset containing legitimate and fraud transactions from the duration 1st Jan 2019 - 31st Dec 2020. It covers credit cards of 10

'Fraud is committed worldwide through various methods, including lost or stolen cards (48%), identity theft (15%), skimming or cloning (14%), counterfeit cards (12%), and mail intercept fraud (6%). The most common types of credit card fraud include application fraud, lost or stolen cards, account takeover, fake and counterfeit cards, merchant collusion, and triangulation. \n\nAccording to available data, the number of fraud committed worldwide is significant, with countries like Ukraine, Indonesia, Yugoslavia, Turkey, and Malaysia being among the high-risk countries facing credit card fraud.'

In [21]:
response

{'messages': [HumanMessage(content='how fraud is committed worldwide? and how many fraud committed?', additional_kwargs={}, response_metadata={}, id='a1d62162-27f8-4504-8c32-3448271b287b'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'sbk4t262r', 'function': {'arguments': '{"query":"how fraud is committed worldwide"}', 'name': 'ask_about_credit_cards_fraud_theory'}, 'type': 'function'}, {'id': 'ea6v3hg3w', 'function': {'arguments': '{"query":"number of fraud committed worldwide"}', 'name': 'ask_about_credit_cards_fraud_data'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 62, 'prompt_tokens': 766, 'total_tokens': 828, 'completion_time': 0.066765013, 'prompt_time': 0.036258625, 'queue_time': 0.208492064, 'total_time': 0.103023638}, 'model_name': 'meta-llama/llama-4-maverick-17b-128e-instruct', 'system_fingerprint': 'fp_c527aa4474', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--ea48d34b-1c1d-4